## SongShu tagging
https://hackmd.io/HBDaoNdVS8aPswHcPnTphw?both

In [1]:
from SongShu import SongShu

In [2]:
%%time
songshu = SongShu(date="2019-11-10", creator="RGT", CBDBpath='C:/NLP/Raft/CBDB_20190424.db')
songshu.load_htmls("./data/medievalHistories/Songshu/")
#songshu.buildDictionaries('./NE_randstr2.txt')
songshu.extract_all()
songshu.encodeTime()

INFO:root:Stop at loading ./data/medievalHistories/Songshu/SongShu_0969.html.
INFO:root:Total length of the data is 969.
ERROR:root:[Error] SongShu_rare_char.json does not exist

            try to run these lines: 
            	>> self.extract_rare_chars()
            	>> self.write_rare_chars()

INFO:root:Remove the new lines added by the page dividers, connect the paragraphs before and after the new lines.
INFO:root:Remove 標註, page number, and page dividers from the tree structure.


Removing bookmarks from main text...
Extracting full person names based on Global NE List...
Getting all person names from bookmarks...
Normalizing all person names from bookmarks...
Combining all person names (global list and bookmarks)...
Wall time: 30.7 s


In [3]:
# Reload sorted global NE list after new entries have been added 
# and BuildGlobalNEList.ipynb has been run.
# songshu.buildDictionaries('./NE_randstr2.txt')

In [4]:
songshu

       type       variable                 method current_length
0      meta      flat_meta      self.extract_meta            969
1      path          paths     self.extract_paths            969
2  passages  flat_passages  self.extract_passages            969

In [5]:
# These are the non-prince biographical entries in Songshu
VALID_BIO_FILENOS = list(range(597,654)) + list(range(664,714)) + list(range(722,751)) + \
                    list(range(755,761)) + list(range(755,761)) + list(range(774, 798))

In [6]:
len(VALID_BIO_FILENOS)

172

In [7]:
%%time
#for fileno in [597, 609, 641, 642, 647, 666, 668, 745]: # VALID_BIO_FILENOS: # 
PROBLEMS = [] # problematic fileno's
for fileno in VALID_BIO_FILENOS:
    #if fileno in [611, 637, 638, 649, 651, 670]: continue # skip these for now
    # Problematic fileno's; need further investigation
    #if fileno in [611, 638, 649, 651, 670, 677, 696, 733, 737, 749, 783] 
    print(fileno)
    print(songshu.dateNodeList(fileno))
    print('-'*60)
    print(songshu.dateList(fileno))
    print('-'*60)
    try:
        soup = songshu.fillMissingDateComponents(fileno, DEBUG=False)
        print(soup)
        print('-'*60)
        print(fr"{songshu.paths_cleaned[fileno]}")
        print(songshu.annotateTime(fileno))
    except:
        print(f"Problem fileno: {fileno}")
        PROBLEMS.append(fileno)
        print(songshu.tag(fileno))
        #continue
    print('='*60)

597
[<time>義熙三年</time>, <time>八年</time>, <time>十年</time>, <time>十一年</time>, <time>十二年</time>, <time>十三年</time>, <time>十一月</time>, <time>元嘉九年</time>, <time>二十五年四月</time>]
------------------------------------------------------------
[['義熙', 3, '', ''], ['', 8, '', ''], ['', 10, '', ''], ['', 11, '', ''], ['', 12, '', ''], ['', 13, '', ''], ['', '', 11, ''], ['元嘉', 9, '', ''], ['', 25, 4, '']]
------------------------------------------------------------
[['義熙', 3, '', ''], ['義熙', 8, '', ''], ['義熙', 10, '', ''], ['義熙', 11, '', ''], ['義熙', 12, '', ''], ['義熙', 13, '', ''], ['義熙', 13, 11, ''], ['元嘉', 9, '', ''], ['元嘉', 25, 4, '']]
------------------------------------------------------------
史／正史／宋書／列傳　凡六十卷／卷四十二　列傳第二／劉穆之
<html><body><p>宋書卷四十二　　列傳第二　　　劉<person id="劉穆之">穆之</person>　王弘劉<person id="劉穆之">穆之</person>，字道和，小字道民，東莞莒人，漢齊悼惠王肥後也。世居京口。少好書、傳，博覽多通，為濟陽江敳所知。敳為建武將軍、琅邪內史，以為府主簿。

初，<person id="劉穆之">穆之</person>嘗夢與高祖俱泛海，忽值大風，驚懼。俯視船下，見有二白龍夾舫。既而至一山，峯㟧聳秀，林樹繁密，意甚悅之。及高祖克京城，問何無忌曰：「急須一府主簿，何由得之？」無忌曰：「無過劉道民

In [8]:
PROBLEMS

[611, 638, 649, 651, 670, 677, 696, 733, 737, 749, 783]

In [9]:
songshu.annotateTime(954)
#songshu.fillMissingDateComponents(648)

ValueError: '永安' is not in list

In [ ]:
def fillMissingDateComponents(book, fileno, DEBUG=False):
    '''
    Purpose:
    Reads dlist (each element of which is a quadruple [era, year, month, day])
    and outputs a "normalized" version, numerical year/month/day data
    is converted into numbers written in Arabic numerals 
    '''
    dlist  = book.dateList(fileno)
    #dlist0 = deepcopy(dlist)

    if len(dlist) <= 1: return dlist # nothing to do

    #lastlastdate = dlist[0]
    #lastdate     = dlist[1] 
    #if lastdate[0] == '':
    #    lastdate[0] == ''
    for idx, d in enumerate(dlist):
        [era, year, month, day] = d
        if idx == 0:
            continue # nothing to do for index 0
        elif idx == 1:
            dlist[idx] = book.updateCurrentDate(dlist[idx], dlist[idx-1], None, DEBUG=DEBUG)
        else: # idx > 1
            dlist[idx] = book.updateCurrentDate(dlist[idx], dlist[idx-1], dlist[idx-2], DEBUG=DEBUG)

    # if the first (0th) element of dlist lacks an era name, fill it in
    # with the *last element from the previous fileno*
    if dlist[0][0] == '': # era missing
        dateListFromPreviousFileno = fillMissingDateComponents(book, fileno-1)
        print(f"Previous fileno {fileno-1}: {dateListFromPreviousFileno}")
        if len(dateListFromPreviousFileno) > 0:
            backup_era = dateListFromPreviousFileno(fileno-1)[-1][0] # from last time node of previous fileno
            if DEBUG: print(f"backup_era = {backup_era}")
            dlist[0][0] = backup_era
        if dlist[0][1] == '': # year missing
            if len(dateListFromPreviousFileno) > 0:
                backup_year = dateListFromPreviousFileno(fileno-1)[-1][1] # same
                if DEBUG: print(f"backup_year = {backup_year}")
                dlist[0][1] = backup_year

    return dlist

In [ ]:
%%time
#for fileno in [597, 609, 641, 642, 647, 666, 668, 745]: # VALID_BIO_FILENOS: # 
PROBLEMS = [] # problematic fileno's
for fileno in VALID_BIO_FILENOS:
    #if fileno in [611, 637, 638, 649, 651, 670]: continue # skip these for now
    # Problematic fileno's; need further investigation
    #if fileno in [611, 638, 649, 651, 670, 677, 696, 733, 737, 749, 783] 
    print(fileno)
    print(songshu.dateNodeList(fileno))
    print('-'*60)
    print(songshu.dateList(fileno))
    print('-'*60)
    try:
        soup = fillMissingDateComponents(songshu, fileno, DEBUG=False)
        print(soup)
        print('-'*60)
        print(fr"{songshu.paths_cleaned[fileno]}")
        print(songshu.annotateTime(fileno))
    except:
        print(f"Problem fileno: {fileno}")
        PROBLEMS.append(fileno)
        print(songshu.tag(fileno))
        #continue
    print('='*60)

In [ ]:
PROBLEMS

In [ ]:
fillMissingDateComponents(songshu, 648)

In [ ]:
for fileno in [637, 638]:
    print(songshu.dateList(fileno))
    print('-'*60)
    print(fillMissingDateComponents(songshu, fileno, DEBUG=False))
    print('='*60)

In [ ]:
songshu.eraNameYear2GregorianYear('義熙', 1, dynasty='東晉', DEBUG=True)


In [ ]:
songshu.eraNameYear2GregorianYear('永嘉', 3, dynasty='西晉', DEBUG=False)

In [ ]:
songshu.eraNameYear2GregorianYear('昇明', 2, dynasty=None, DEBUG=True)

In [ ]:
era = '隆安' # '元熙'
songshu.eraNameYear2GregorianYear(era, 4, dynasty=None, DEBUG=True)

In [ ]:
songshu.annotateTime(883)

In [ ]:
from SongShu import regex_date

In [ ]:
regex_date

In [ ]:
dlist  = songshu.dateList(611)
dlist

In [ ]:
len(PROBLEMS)/len(VALID_BIO_FILENOS)

In [ ]:
PROBLEMS

In [ ]:
songshu.fillMissingDateComponents(611)

In [ ]:
songshu.annotateTime(611)